In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 61.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import global_add_pool
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import GIN_MLP_Layers as gin_mlp_layers
from scipy.sparse import csr_matrix
from sklearn import metrics
from time import perf_counter

In [56]:
class GIN_MLPs(nn.Module):
    def __init__(self, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_act_fun, Bias):
        super(GIN_MLPs, self).__init__()
        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.num_slp_layers = num_slp_layers
        self.Bias = Bias

        if mlp_act_fun == 'ReLu':
            self.mlp_act_fun = F.relu
        elif mlp_act_fun == 'eLu':
            self.mlp_act_fun = nn.functional.elu
        elif mlp_act_fun == 'tanh':
            self.mlp_act_fun = torch.tanh

        self.gin_mlp_layers = torch.nn.ModuleList()
        self.gin_batch_normalization = torch.nn.ModuleList()


        if self.num_slp_layers == 1:
            self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
            self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))

        elif self.num_slp_layers > 1:
            for i in range(self.num_slp_layers):
                if i == 0:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))

                else:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))
        else:
            print("please enter layer config")

    def forward(self, h):
        for i in range(self.num_slp_layers):
            layer = self.gin_mlp_layers[i](h)
            bnorm = self.gin_batch_normalization[i](layer)
            h = self.mlp_act_fun(bnorm)
        return h

x_inp_dim = 7
mlp_example = GIN_MLPs(num_slp_layers=2, mlp_input_dim=x_inp_dim, mlp_hid_dim=10, mlp_act_fun="ReLu", Bias=True)
x = torch.rand(20,x_inp_dim)
#x = x.permute(0, 2, 1)
y = mlp_example(x)
print(y.size())


torch.Size([20, 10])


In [ ]:
"""class GIN_MLPs(nn.Module):
    def __init__(self, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, Bias):
        super(GIN_MLPs, self).__init__()
        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.Bias = Bias

        if mlp_act_fun == 'ReLu':
            self.mlp_act_fun = F.relu
        elif mlp_act_fun == 'eLu':
            self.mlp_act_fun = nn.functional.elu
        elif mlp_act_fun == 'tanh':
            self.mlp_act_fun = torch.tanh

        self.gin_mlp_layers = torch.nn.ModuleList()
        self.gin_batch_normalization = torch.nn.ModuleList()

        if self.num_slp_layers == 1:
            self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_output_dim, out_features=self.mlp_output_dim, bias=self.Bias))
            self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
        elif self.num_slp_layers > 1:
            for i in range(self.num_slp_layers):
                if i == 0:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_output_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(num_features=self.mlp_hid_dim))
                elif i == self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_output_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
                elif 0 < i < self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))
        else:
            print("please enter layer config")

    def forward(self, h):

        for i in range(self.num_slp_layers):
            if i == 0:
                layer = self.gin_mlp_layers[i](h)
                layer = layer.permute(0, 2, 1)
                bnorm = self.gin_batch_normalization[i](layer)
                bnorm = bnorm.permute(0, 2, 1)
                h = self.mlp_act_fun(bnorm)
            else:
                h = self.mlp_act_fun(self.gin_mlp_layers[i](h))
        return h

x_inp_dim = 7
mlp_example = GIN_MLPs(num_slp_layers=2, mlp_input_dim=x_inp_dim, mlp_hid_dim=17, mlp_output_dim=7, mlp_act_fun="ReLu", Bias=True)
x = torch.rand(1,20,x_inp_dim)
#x = x.permute(0, 2, 1)
y = mlp_example(x)
print(y.size())
"""

In [ ]:
# With Learnable Parameters
#m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(7, affine=False)
input = torch.randn(300, 100, 7)
input = input.permute(0, 2, 1)
output = m(input)
output = output.permute(0, 2, 1)
print(output.size())

In [44]:
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################

In [ ]:
global_sum_pool = GlobalSUMPool()

In [ ]:
x = torch.arange(140)
x = x.view(2,10, 7)
print(x)
x = x.permute(0,1,2)
print(x)

In [57]:
from IPython.core.display import deepcopy
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch_geometric
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
#from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import global_add_pool
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import GIN_MLP_Layers as gin_mlp_layers


class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, dropout_rate,
                 Weight_Initializer, joint_embeddings):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias
        self.joint_embeddings = joint_embeddings

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers))
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.the_first_layer = nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_hid_dim, bias=self.Bias)
        #if self.joint_embeddings:
        #    self.lin1 = nn.Linear(in_features=self.mlp_hid_dim * (self.num_mlp_layers + 1), out_features=self.mlp_hid_dim * (self.num_mlp_layers + 1))
        #    self.lin2 = nn.Linear(in_features=self.mlp_hid_dim * (self.num_mlp_layers + 1), out_features=self.mlp_output_dim)
        #else:
        self.lin1 = nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_hid_dim)
        self.lin2 = nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_output_dim)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_hid_dim,
                                                mlp_hid_dim=self.mlp_hid_dim, mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, h, batched_graphs, edge_mask):

        #joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())# + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
        if batched_graphs.batch == None:
            batch_size = 1
        else:
            batch_size = batched_graphs.num_graphs

        pooled = torch.mm(joint_tilda_adjacency_matrix, h)


        return joint_tilda_adjacency_matrix, pooled


    def gin_layer_process_eps(self, h, layer, batched_graphs, edge_mask):

        joint_tilda_adjacency_matrix, pooled = self.gin_neighborhood_aggregation(h, batched_graphs, edge_mask)

        pooled = pooled + (1 + self.eps[layer])*h
        pooled_rep = self.gin_mlp_layers[layer](pooled)


        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds


    def forward(self, batched_graphs, edge_mask):

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        else:
            graph_sizes = [len(batched_graphs.x)]

        mlps_output_embeds = []
        mlps_output_embeds_pooled = []
        hid_rep = self.the_first_layer(batched_graphs.x)
        mlps_output_embeds.append(hid_rep)

        for layer in range(self.num_mlp_layers):
            hid_rep = self.gin_layer_process_eps(hid_rep, layer, batched_graphs, edge_mask)
            mlps_output_embeds.append(hid_rep)

        mlp_outputs_globalSUMpooled = 0
        if self.joint_embeddings:
            for mlp_output in mlps_output_embeds:
                mlp_outputs_globalSUMpooled += self.global_summing(mlp_output, batched_graphs.batch)
        else:
            mlp_outputs_globalSUMpooled = self.global_summing(hid_rep, batched_graphs.batch)

        lin1_output = self.lin1(mlp_outputs_globalSUMpooled)
        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=-1)

        return mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed



mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batch_size = 20
node_feat_size = len(mutag_dataset[0].x[0])
mutag_train_dataset = mutag_dataset[:150]
mutag_test_dataset = mutag_dataset[150:]
mutag_batched_train_dataloader = DataLoader(mutag_train_dataset, batch_size=batch_size, shuffle=True)
mutag_batched_test_dataloader = DataLoader(mutag_test_dataset, batch_size=batch_size, shuffle=True)
gin_model_example = GIN_Model(num_mlp_layers=4, Bias=True, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=8,
                              mlp_output_dim=2, mlp_act_fun="ReLu", dropout_rate=0.5, Weight_Initializer=3, joint_embeddings=False)
#print(gin_model_example)

for batched_graphs in mutag_batched_train_dataloader:
    #x, edge_index, batch, y = batched_graphs.x, batched_graphs.edge_index, batched_graphs.batch, batched_graphs.y
    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = gin_model_example(batched_graphs, None)
    print("lin2_output_softmaxed: ", lin2_output_softmaxed.size())
    print(lin2_output_softmaxed)
    break

lin2_output_softmaxed:  torch.Size([20, 2])
tensor([[0.5915, 0.4085],
        [0.6043, 0.3957],
        [0.4643, 0.5357],
        [0.3353, 0.6647],
        [0.7954, 0.2046],
        [0.5611, 0.4389],
        [0.7308, 0.2692],
        [0.6453, 0.3547],
        [0.1647, 0.8353],
        [0.6432, 0.3568],
        [0.5698, 0.4302],
        [0.6988, 0.3012],
        [0.4305, 0.5695],
        [0.6571, 0.3429],
        [0.7512, 0.2488],
        [0.6883, 0.3117],
        [0.3994, 0.6006],
        [0.6057, 0.3943],
        [0.7891, 0.2109],
        [0.6881, 0.3119]], grad_fn=<SoftmaxBackward0>)


In [58]:
GNN_Model_optimizer = torch.optim.Adam(gin_model_example.parameters(), lr=0.001, weight_decay=1e-4)

In [59]:
for batched_graphs in mutag_batched_train_dataloader:
    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = gin_model_example(batched_graphs, None)
    print("lin2_output_softmaxed: ", lin2_output_softmaxed.size())
    print(lin2_output_softmaxed)
    break

lin2_output_softmaxed:  torch.Size([20, 2])
tensor([[0.5595, 0.4405],
        [0.7480, 0.2520],
        [0.5598, 0.4402],
        [0.5317, 0.4683],
        [0.6762, 0.3238],
        [0.5814, 0.4186],
        [0.3324, 0.6676],
        [0.6984, 0.3016],
        [0.2218, 0.7782],
        [0.5346, 0.4654],
        [0.6427, 0.3573],
        [0.4228, 0.5772],
        [0.6610, 0.3390],
        [0.3636, 0.6364],
        [0.0402, 0.9598],
        [0.5698, 0.4302],
        [0.5721, 0.4279],
        [0.5876, 0.4124],
        [0.5397, 0.4603],
        [0.6489, 0.3511]], grad_fn=<SoftmaxBackward0>)


In [60]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch


In [61]:
def train_step(data):
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    gin_model_example.train()
    gin_model_example.zero_grad()
    #torch.autograd.set_detect_anomaly(True)
    for batch_of_graphs in data:

        mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, softmaxed_h2 = gin_model_example(batch_of_graphs, None)
        batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)

        #print(softmaxed_h2, softmaxed_h2.argmax(dim=1))
        Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())
        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()
        #print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])
        GNN_Model_optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels), gin_model_example


In [62]:
#from IPython.display import Javascript  # Restrict height of output cell.
#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


SA_Model_training_time_per_epoch = []
SA_Model_training_Acc_per_epoch = []

def train(EPOCHS, load_index, data):
    SA_training_loss_per_epoch = []

    for epoch in range(EPOCHS):

        start_generation = perf_counter()
        SA_model_training_loss, training_acc, gin_model_example = train_step(data)
        SA_Model_training_time_per_epoch.append(perf_counter() - start_generation)
        SA_Model_training_Acc_per_epoch.append(training_acc)

        print(f'Epoch: {epoch+1:03d}, Model Loss: {SA_model_training_loss:.4f} Accuracy: {training_acc}')

        SA_training_loss_per_epoch.append(SA_model_training_loss)


        #if (epoch + load_index + 1) % Visualization_Parameter == 0 and epoch > 0:
        #    visualize_losses(SA_training_loss_per_epoch, epoch + load_index + 1)
        #if (epoch + load_index + 1) % Model_Saving_Parameter == 0 and epoch > 0:
        #    torch.save({'epoch': epoch+load_index+1, 'model_state_dict': GNN_Model.state_dict(), 'optimizer_state_dict': GNN_Model_optimizer.state_dict(), 'loss': SA_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1)+".pt")
    return gin_model_example



In [63]:
print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])

tensor([[-0.1597, -0.2933, -0.0153,  0.0976,  0.3306, -0.0476,  0.0958,  0.2779],
        [ 0.2149,  0.0996,  0.0077,  0.2491,  0.0335,  0.1778, -0.0147,  0.3481],
        [ 0.0018,  0.0817, -0.0168, -0.2734,  0.3361,  0.2240,  0.2341, -0.0358],
        [-0.1962, -0.2848, -0.0325,  0.1340,  0.2170,  0.0180,  0.0260, -0.3439],
        [ 0.2158,  0.3150,  0.0214,  0.0277, -0.3298,  0.1835,  0.1944, -0.2944],
        [ 0.0066, -0.0089,  0.2982,  0.2753, -0.1625,  0.2939,  0.1651, -0.1186],
        [ 0.1905, -0.1403, -0.1574,  0.2863,  0.0099, -0.2352,  0.0642, -0.0143],
        [-0.1220, -0.2692, -0.1532, -0.2907, -0.1294, -0.0472,  0.1681, -0.0118]])


In [64]:
EPOCHS = 300
load_index = 0

gin_model_example = train(EPOCHS, load_index, mutag_batched_train_dataloader)

Epoch: 001, Model Loss: 0.7301 Accuracy: 0.4533333333333333
Epoch: 002, Model Loss: 0.7029 Accuracy: 0.5266666666666666
Epoch: 003, Model Loss: 0.6306 Accuracy: 0.5533333333333333
Epoch: 004, Model Loss: 0.6336 Accuracy: 0.62
Epoch: 005, Model Loss: 0.6163 Accuracy: 0.6266666666666667
Epoch: 006, Model Loss: 0.6131 Accuracy: 0.6133333333333333
Epoch: 007, Model Loss: 0.5831 Accuracy: 0.64
Epoch: 008, Model Loss: 0.5857 Accuracy: 0.66
Epoch: 009, Model Loss: 0.5783 Accuracy: 0.6666666666666666
Epoch: 010, Model Loss: 0.5438 Accuracy: 0.72
Epoch: 011, Model Loss: 0.5477 Accuracy: 0.6933333333333334
Epoch: 012, Model Loss: 0.5338 Accuracy: 0.7333333333333333
Epoch: 013, Model Loss: 0.5357 Accuracy: 0.76
Epoch: 014, Model Loss: 0.5023 Accuracy: 0.7866666666666666
Epoch: 015, Model Loss: 0.4898 Accuracy: 0.8266666666666667
Epoch: 016, Model Loss: 0.4952 Accuracy: 0.84
Epoch: 017, Model Loss: 0.4787 Accuracy: 0.8466666666666667
Epoch: 018, Model Loss: 0.5120 Accuracy: 0.7866666666666666
Epoc

In [65]:
print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])

tensor([[-0.2844, -0.3623, -0.1200,  0.1063,  0.2084, -0.1380,  0.0061,  0.2829],
        [ 0.1412,  0.1092,  0.0201,  0.3368,  0.1260,  0.3107,  0.0436,  0.3992],
        [-0.0406,  0.0394, -0.0252, -0.2404,  0.3206,  0.2871,  0.3359, -0.0339],
        [-0.2699, -0.2893, -0.0423,  0.1710,  0.2888,  0.0964,  0.1118, -0.2965],
        [ 0.1566,  0.4170,  0.1765, -0.1816, -0.2968,  0.0655,  0.0974, -0.3794],
        [-0.0023,  0.0654,  0.3780,  0.1698, -0.0272,  0.2466,  0.2234, -0.1896],
        [ 0.2285, -0.2109, -0.2160,  0.2672, -0.1464, -0.2578, -0.0213, -0.0217],
        [-0.1469, -0.4470, -0.2574, -0.2453, -0.1932, -0.0271,  0.1182,  0.0147]])


In [68]:
GNN_Model_test_predicted_labels = []
def GNN_Model_test(test_loader):
    gin_model_example.eval()
    correct = 0
    for batch_of_graphs in test_loader:
        mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = gin_model_example(batch_of_graphs, None)
        GNN_Model_test_pred = soft.argmax(dim=1)

        GNN_Model_test_predicted_labels.append(GNN_Model_test_pred.tolist()[0])
        correct += int((GNN_Model_test_pred == batch_of_graphs.y).sum())
    return correct / len(test_loader), GNN_Model_test_predicted_labels

In [69]:
from sklearn.metrics import roc_auc_score
GNN_Model_test_acc, predicted_labels = GNN_Model_test(mutag_test_dataset)
print(f'Test Accuracy: {GNN_Model_test_acc:.4f}')
print(predicted_labels)
real_labels = []
for graph in mutag_test_dataset:
    real_labels.append(graph.y.tolist()[0])
print(real_labels)
micro_roc_auc_ovr = roc_auc_score(np.array(real_labels), np.array(predicted_labels), multi_class="ovr", average="micro")
micro_roc_auc_ovr = roc_auc_score(np.array(real_labels), np.array(predicted_labels), multi_class="ovr", average="micro")
print("micro_roc_auc_ovr: ", micro_roc_auc_ovr)

Test Accuracy: 0.7895
[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
micro_roc_auc_ovr:  0.7564102564102566


In [ ]:
'''
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import global_add_pool
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import GIN_MLP_Layers as gin_mlp_layers


class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, num_classes, dropout_rate, Weight_Initializer):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers))
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.lin1 = nn.Linear(in_features=self.mlp_output_dim * (self.num_mlp_layers + 1), out_features=self.mlp_input_dim * (self.num_mlp_layers + 1))
        self.lin2 = nn.Linear(in_features=self.mlp_input_dim * (self.num_mlp_layers + 1), out_features=self.num_classes)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(gin_mlp_layers.GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_input_dim, mlp_hid_dim=self.mlp_hid_dim, mlp_output_dim=self.mlp_output_dim, mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, h, batched_graphs, edge_mask):

        #joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())# + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
        if batched_graphs.batch == None:
            batch_size = 1
        else:
            batch_size = batched_graphs.num_graphs

        pooled = torch.mm(joint_tilda_adjacency_matrix, batched_graphs.x)


        return joint_tilda_adjacency_matrix, pooled


    def gin_layer_process_eps(self, h, layer, batched_graphs, edge_mask):

        joint_tilda_adjacency_matrix, pooled = self.gin_neighborhood_aggregation(h, batched_graphs, edge_mask)

        pooled = pooled + (1 + self.eps[layer])*h
        pooled_rep = self.gin_mlp_layers[layer](pooled)


        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds





    def forward(self, batched_graphs, edge_mask):
        #X_concatinated = [graph for graph in batched_graphs.x]
        #X_concatinated = torch.stack(X_concatinated, dim=0)

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        else:
            graph_sizes = [len(batched_graphs.x)]

        mlps_output_embeds = []
        mlps_output_embeds.append(batched_graphs.x)
        hid_rep = batched_graphs.x

        for layer in range(self.num_mlp_layers):
            hid_rep = self.gin_layer_process_eps(hid_rep, layer, batched_graphs, edge_mask)
            mlps_output_embeds.append(hid_rep)
        #print("the last mlp's: ",h.size())
        #h = torch.split(hid_rep, graph_sizes)


        mlps_output_embeds_stacked = torch.stack(mlps_output_embeds)

        mlp_outputs_globalSUMpooled = self.global_summing(mlps_output_embeds_stacked, batched_graphs.batch)

        #merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        if batched_graphs.batch == None:
            #merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlps_output_embeds, graph_sizes, batched_graphs.batch)
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(torch.unsqueeze(mlp_outputs_globalSUMpooled, dim=1), graph_sizes)
        else:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        lin1_output = self.lin1(torch.tensor(merged_mlps_output_embeds_reshaped))
        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=1)

        return mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed



#dataset = TUDataset(root='data/TUDataset', name='MUTAG')
#batch_size = 20
#node_feat_size = len(dataset[0].x[0])
#batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)
#gin_model_example = GIN_Model(num_mlp_layers=4, Bias=True, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=7, mlp_output_dim=7, #mlp_act_fun="ReLu", num_classes=2, dropout_rate=0.5, Weight_Initializer=3)
#print(gin_model_example)

#for batched_graphs in batched_dataset:
#    #x, edge_index, batch, y = batched_graphs.x, batched_graphs.edge_index, batched_graphs.batch, batched_graphs.y
#    mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, #lin1_output_dropouted, lin2_output, lin2_output_softmaxed = gin_model_example(batched_graphs)
#    print("lin2_output_softmaxed: ", lin2_output_softmaxed.size())
#    print(lin2_output_softmaxed)
#    break'''

GIN_Model(
  (gin_mlp_layers): ModuleList(
    (0-3): 4 x GIN_MLPs(
      (gin_mlp_layers): ModuleList(
        (0-1): 2 x Linear(in_features=7, out_features=7, bias=True)
      )
      (gin_batch_normalization): ModuleList(
        (0): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (global_summing): GlobalSUMPool()
  (lin1): Linear(in_features=35, out_features=35, bias=True)
  (lin2): Linear(in_features=35, out_features=2, bias=True)
  (dorpout): Dropout(p=0.5, inplace=False)
)


In [ ]:
'''from IPython.core.display import deepcopy
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, num_classes,
                 dropout_rate, Weight_Initializer):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers))
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.lin1 = nn.Linear(in_features=self.mlp_output_dim * (self.num_mlp_layers + 1), out_features=self.mlp_input_dim * (self.num_mlp_layers + 1))
        self.lin2 = nn.Linear(in_features=self.mlp_input_dim * (self.num_mlp_layers + 1), out_features=self.num_classes)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_input_dim,
                                                               mlp_hid_dim=self.mlp_hid_dim, mlp_output_dim=self.mlp_output_dim,
                                                               mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, new_adjacecny, new_features):

        pooled = torch.bmm(new_adjacecny, new_features)


        return pooled


    def gin_layer_process_eps(self, hid_rep, layer, new_adjacecny, new_features):

        pooled = self.gin_neighborhood_aggregation(new_adjacecny, new_features)


        pooled = pooled + (1 + self.eps[layer])*hid_rep
        pooled_rep = self.gin_mlp_layers[layer](pooled)


        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds


    def computational_matrix(self, batched_graphs, edge_mask):

        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)


        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1
        max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)


        adjacency_list = []
        feature_list = []
        for i in range(batch_size):
            start = i * graph_sizes[i]
            end = (i + 1) * graph_sizes[i]
            un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
            off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
            if un_padded_adj.size()[0] <= max_number_of_nodes_in_batch_of_graphs:
                un_padded_adj = F.pad(un_padded_adj, (0, off_set, 0, off_set), mode='constant', value=0)
                un_padded_adj = un_padded_adj.type(torch.float32)

            un_padded_adj = un_padded_adj.type(torch.float32)
            adjacency_list.append(un_padded_adj)

            un_padded_feat = batched_graphs.x[start:end, :]
            un_padded_feat = F.pad(un_padded_feat, (0, 0, 0, off_set), mode='constant', value=0)
            un_padded_feat = un_padded_feat.type(torch.float32)
            un_padded_feat.require_grad = True
            feature_list.append(un_padded_feat)

        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)

        return new_adjacecny, new_features


    def forward(self, batched_graphs, edge_mask):

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        else:
            graph_sizes = [len(batched_graphs.x)]
        batched_graphs2 = deepcopy(batched_graphs)
        print(batched_graphs.batch, batched_graphs.batch.size())
        new_adjacecny, new_features = self.computational_matrix(batched_graphs, edge_mask)
        mlps_output_embeds = []
        mlps_output_embeds.append(new_features)



        source = new_features
        for layer in range(self.num_mlp_layers):
            new_features = self.gin_layer_process_eps(new_features, layer, new_adjacecny, new_features)
            source = torch.cat((source, new_features),0)
            mlps_output_embeds.append(new_features)

        batch_list = []
        for i in range(source.size()[0]):
            for j in range(source.size()[1]):
                batch_list.append(i)
        batch_np = np.array(batch_list)
        batch_torch = torch.from_numpy(batch_np)
        source = source.view(source.size()[0]*source.size()[1], source.size()[2])
        print("here", source.size())
        print("new_features: ", new_features.size())


        #mlps_output_embeds_stacked = torch.stack(mlps_output_embeds)
        #print("mlps_output_embeds_stacked: ", mlps_output_embeds_stacked.size())

        mlp_outputs_globalSUMpooled = self.global_summing(source, batch_torch)
        print("mlp_outputs_globalSUMpooled: ", mlp_outputs_globalSUMpooled.size())

        if batched_graphs.batch == None:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(torch.unsqueeze(mlp_outputs_globalSUMpooled, dim=1), graph_sizes)
        else:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        lin1_output = self.lin1(torch.tensor(merged_mlps_output_embeds_reshaped))
        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=1)

        return mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed
node_feat_size=7
gin_model_example = GIN_Model(num_mlp_layers=4, Bias=True, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=7,
                              mlp_output_dim=7, mlp_act_fun="ReLu", num_classes=2, dropout_rate=0.5, Weight_Initializer=3)
#print(gin_model_example)

'''